In [1]:
from bs4 import BeautifulSoup
import datetime, time
import requests
import pandas as pd
import os.path
from os import path

C:\Users\Deepak\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
def getTickers():
    """Returns the tickers for all the S&P500 companies using the Wikipedia page
    Outputs: 
        tickers - list of tickers for every company in the S&P500
    """
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find("table") # tickers are contained in a table
    tickers = []
    for row in table.find_all('tr'):
            cols = row.find_all('td')
            if cols:
                tickers.append(cols[0].text.strip())
    return tickers

In [3]:
def voltest(ticker):
    url = "https://www.alphaquery.com/stock/"+ ticker+ "/volatility-option-statistics/30-day/iv-mean"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    indicator_row = soup.find('tr', id='indicator-iv-mean')
    indicator_value = indicator_row.find('div', class_='indicator-figure-inner').text
    return float(indicator_value)

In [4]:
def getStockVol(ticker):
    """Returns a stock's 30-day implied volatility from alphaqueries
    Inputs:
        ticker     - a string representing a stock's ticker
    Outputs: 
        volatility - implied volatility for the stock 
    """
        
    url = "https://www.alphaquery.com/stock/"+ ticker+ "/volatility-option-statistics/30-day/iv-mean"
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    indicator_row = soup.find('tr', id='indicator-iv-mean')
    indicator_value = indicator_row.find('div', class_='indicator-figure-inner').text
    if indicator_value == '--':
        return False
    
    return float(indicator_value)

In [5]:
stocks = getTickers()
for i in stocks:
    print(i)
    if getStockVol(i) is False :
        print('remove', i)
        stocks.remove(i)
print(stocks)

MMM
AOS
ABT
ABBV
ACN
ATVI
ADM
ADBE
ADP
AAP
AES
AFL
A
APD
AKAM
ALK
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AMD
AEE
remove AEE
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ANET
AJG
AIZ
T
ATO
ADSK
AZO
AVB
AVY
AXON
BKR
BALL
BAC
BBWI
BAX
BDX
WRB
BRK.B
BBY
BIO
TECH
BIIB
BLK
BK
BA
BKNG
BWA
remove BWA
BSX
BMY
AVGO
BR
BRO
BF.B
BG
CHRW
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
CNC
CNP
CDAY
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CTVA
remove CTVA
COST
CTRA
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DXCM
FANG
DLR
DFS
DIS
DG
DLTR
D
DPZ
DOV
DOW
DTE
DUK
DD
DXC
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
LLY
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ESS
EL
ETSY
EG
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FICO
FAST
FRT
FDX
FITB
FSLR
FE
FIS
FI
FLT
FMC
F
FTNT
FTV
FOXA
FOX
BEN
FCX
GRMN
IT
GEHC
GEN
GNRC
GD
GE
GIS
GM
GPC
GILD
GL
remove GL
GS
HAL
HIG
HAS
HCA
PEAK
HSIC
HSY
HES
H

In [43]:
def getStockData(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # Get stock price
    price_element = soup.find("td", {"data-test": "ASK-value"})
    stock_price = price_element.text
    x = stock_price.find('x')
    stock_price = float(stock_price[0:x].strip().replace(',',''))

    # Get dividend yield
    div_yield_element = soup.find("td", {"data-test": "DIVIDEND_AND_YIELD-value"})
    div_yield = div_yield_element.text if div_yield_element else "N/A"
    
    x = div_yield.find('(')
    if "N" not in div_yield[x+1:-2]: # Only set dividend if not 'N/A'
        div_yield = float(div_yield[x+1:-2])/100
    else: 
        div_yield = 0

    # Get implied volatility
    ticker = ticker.replace("-",".") #BRK.B exception
    implied_volatility = getStockVol(ticker)

    return stock_price, div_yield, implied_volatility

In [44]:
getStockData('AAPL')

(179.9, 0.0054, 0.2018)

## Import Option Dates

In [45]:
import yfinance as yf

def get_options_data(ticker):
    stock = yf.Ticker(ticker)
    options = stock.options
    return list(options)

# Usage example
get_options_data('AAPL')

['2023-09-01',
 '2023-09-08',
 '2023-09-15',
 '2023-09-22',
 '2023-09-29',
 '2023-10-06',
 '2023-10-20',
 '2023-11-17',
 '2023-12-15',
 '2024-01-19',
 '2024-02-16',
 '2024-03-15',
 '2024-04-19',
 '2024-06-21',
 '2024-09-20',
 '2024-12-20',
 '2025-01-17',
 '2025-06-20',
 '2025-12-19']

In [46]:
import datetime
import yfinance as yf

def get_options_unix_dates(ticker):
    stock = yf.Ticker(ticker)
    options_dates = stock.options

    unix_dates = []
    for date_string in options_dates:
        date = datetime.datetime.strptime(date_string, "%Y-%m-%d")
        unix_date = int(date.timestamp())
        unix_dates.append(unix_date)

    return unix_dates

# Usage example
ticker_symbol = "AAPL"
unix_dates = get_options_unix_dates(ticker_symbol)

print(unix_dates)


[1693522800, 1694127600, 1694732400, 1695337200, 1695942000, 1696546800, 1697756400, 1700179200, 1702598400, 1705622400, 1708041600, 1710460800, 1713481200, 1718924400, 1726786800, 1734652800, 1737072000, 1750374000, 1766102400]


## Import Options Chain

In [62]:
import yfinance as yf
import pandas as pd

def pick_call_price(row):
    if row['ask'] != 0 and row['bid'] != 0:
        value = (row['ask'] + row['bid']) / 2 #Find the mid of ask and bid price
        return round(value,2)    
    
    elif row['ask'] != 0:
        return row['ask']
    
    elif row['bid'] != 0:
        return row['bid']        
    
    else:
        return 0


def get_strike_and_call_values(ticker, expiration_date):
    stock = yf.Ticker(ticker)
    options_chain = stock.option_chain(expiration_date)
    calls = options_chain.calls

    current_stock_price, div_yield, implied_volatility = getStockData(ticker)

    calls['call'] = calls.apply(pick_call_price, axis=1)
    calls = calls[(calls['strike'] <= current_stock_price) & (calls['volume'] > 0)]  # Filter in-the-money calls with volume
    strikes = calls['strike'].tolist()
    call_values = calls['call'].tolist()

    return strikes, call_values

# Usage example
ticker_symbol = "AAPL"
exp_dates = get_options_data(ticker_symbol)  # Specify the desired expiration date in YYYY-MM-DD format

strikes, call_values = get_strike_and_call_values(ticker_symbol, exp_dates[0])

# Print strikes and call values
print("Strike Prices:", strikes)
print("Call Values:", call_values)


Strike Prices: [85.0, 100.0, 105.0, 125.0, 130.0, 135.0, 140.0, 145.0, 150.0, 152.5, 155.0, 157.5, 160.0, 162.5, 165.0, 167.5, 170.0, 172.5, 175.0, 177.5]
Call Values: [94.8, 79.5, 74.22, 54.52, 49.8, 44.58, 39.35, 34.42, 29.48, 27.1, 24.57, 22.05, 19.6, 17.02, 14.55, 12.1, 9.57, 7.28, 4.93, 2.94]


In [63]:
def Scrapping(startIndex, bs, rf, wait, verbose=True):
    cols = ['Stock Price', 'Strike Price', 'Maturity', 'Dividends', 'Volatility', 'Risk-free', 'Call Price']
    results = pd.DataFrame(columns=cols)
    RISK_FREE = rf
    stock_names = []
    
    # Calculate unixToday
    unixToday = int(time.time())
    
    for i, ticker in enumerate(stocks[startIndex:startIndex+bs]):
        ticker = ticker.replace(".", "-")  # BRK.B exception
        frame = pd.DataFrame(columns=cols)  # Create a fresh frame

        if verbose:
            print(ticker, (i+startIndex))

        # Get stock data
        stock_price, div_yield, volatility = getStockData(ticker)

        # Start option extraction
        url = "https://finance.yahoo.com/quote/"+ticker+"/options"
        dates = get_options_data(ticker)
        
        strikes, call_prices = get_strike_and_call_values(ticker, dates[0])
        unix_dates = get_options_unix_dates(ticker)

        if strikes is None:  # Avoid crashes with empty options page
            pass

        maturity = (unix_dates[0] - unixToday) / (60*60*24*365.25)  # Convert UNIX time difference to fraction of a year
        if maturity <= 0:
            maturity = 1e-5  # Trivial maturity for options that expire today

        # Filter in-the-money calls
        in_the_money_indices = [idx for idx, strike in enumerate(strikes) if strike <= stock_price]
        strikes = [strikes[idx] for idx in in_the_money_indices]
        call_prices = [call_prices[idx] for idx in in_the_money_indices]

        # Insert data into the frame
        frame['Strike Price'] = strikes
        frame['Call Price'] = call_prices
        frame['Stock Price'] = stock_price
        frame['Dividends'] = div_yield
        frame['Volatility'] = volatility
        frame['Risk-free'] = RISK_FREE
        frame['Maturity'] = maturity

        stock_names.extend([ticker] * len(frame.index))
        results = pd.concat([results, frame])  # Concatenate the frame to the results DataFrame

        for date, unix_date in zip(dates[1:], unix_dates[1:]):
            frame = pd.DataFrame(columns=cols)
            maturity = (unix_date - unixToday) / (60*60*24*365.25)  # Convert UNIX time difference to fraction of a year

            strikes, call_prices = get_strike_and_call_values(ticker, date)
            # Add a sleep statement here if needed
            unix_dates = get_options_unix_dates(ticker)

            # Filter in-the-money calls
            in_the_money_indices = [idx for idx, strike in enumerate(strikes) if strike <= stock_price]
            strikes = [strikes[idx] for idx in in_the_money_indices]
            call_prices = [call_prices[idx] for idx in in_the_money_indices]

            # Add data to the frame
            frame['Strike Price'] = strikes
            frame['Call Price'] = call_prices
            frame['Stock Price'] = stock_price
            frame['Dividends'] = div_yield
            frame['Volatility'] = volatility
            frame['Risk-free'] = RISK_FREE
            frame['Maturity'] = maturity

            stock_names.extend([ticker] * len(frame.index))
            results = pd.concat([results, frame])  # Concatenate the frame to the results DataFrame

        print('Complete', i+1)

    results['Stock Name'] = stock_names

    return results


In [66]:
#First 50 stocks

bs = 50
rf = 0.0423 #Current 10 year US treasury bond yield
wait_period = 5
verbose = True
startIdx = 0
    
batch_1 = Scrapping(startIdx, bs, rf, wait_period, verbose)

MMM 0
Complete 1
AOS 1
Complete 2
ABT 2
Complete 3
ABBV 3
Complete 4
ACN 4
Complete 5
ATVI 5
Complete 6
ADM 6
Complete 7
ADBE 7
Complete 8
ADP 8
Complete 9
AAP 9
Complete 10
AES 10
Complete 11
AFL 11
Complete 12
A 12
Complete 13
APD 13
Complete 14
AKAM 14
Complete 15
ALK 15
Complete 16
ALB 16
Complete 17
ARE 17
Complete 18
ALGN 18
Complete 19
ALLE 19
Complete 20
LNT 20
Complete 21
ALL 21
Complete 22
GOOGL 22
Complete 23
GOOG 23
Complete 24
MO 24
Complete 25
AMZN 25
Complete 26
AMCR 26
Complete 27
AMD 27
Complete 28
AAL 28
Complete 29
AEP 29
Complete 30
AXP 30
Complete 31
AIG 31
Complete 32
AMT 32
Complete 33
AWK 33
Complete 34
AMP 34
Complete 35
ABC 35
Complete 36
AME 36
Complete 37
AMGN 37
Complete 38
APH 38
Complete 39
ADI 39
Complete 40
ANSS 40
Complete 41
AON 41
Complete 42
APA 42
Complete 43
AAPL 43
Complete 44
AMAT 44
Complete 45
APTV 45
Complete 46
ACGL 46
Complete 47
ANET 47
Complete 48
AJG 48
Complete 49
AIZ 49
Complete 50


In [68]:
#Batch 2

bs = 50
rf = 0.0423 #Current 10 year US treasury bond yield
wait_period = 5
verbose = True
startIdx = 51
    
batch_2 = Scrapping(startIdx, bs, rf, wait_period, verbose)

ATO 51
Complete 1
ADSK 52
Complete 2
AZO 53
Complete 3
AVB 54
Complete 4
AVY 55
Complete 5
AXON 56
Complete 6
BKR 57
Complete 7
BALL 58
Complete 8
BAC 59
Complete 9
BBWI 60
Complete 10
BAX 61
Complete 11
BDX 62
Complete 12
WRB 63
Complete 13
BRK-B 64
Complete 14
BBY 65
Complete 15
BIO 66
Complete 16
TECH 67
Complete 17
BIIB 68
Complete 18
BLK 69
Complete 19
BK 70
Complete 20
BA 71
Complete 21
BKNG 72
Complete 22
BXP 73
Complete 23
BSX 74
Complete 24
BMY 75
Complete 25
AVGO 76
Complete 26
BR 77
Complete 27
BRO 78
Complete 28
BF-B 79
Complete 29
BG 80
Complete 30
CHRW 81
Complete 31
CDNS 82
Complete 32
CZR 83
Complete 33
CPT 84
Complete 34
CPB 85
Complete 35
COF 86
Complete 36
CAH 87
Complete 37
KMX 88
Complete 38
CCL 89
Complete 39
CARR 90
Complete 40
CTLT 91
Complete 41
CAT 92
Complete 42
CBOE 93
Complete 43
CBRE 94
Complete 44
CDW 95
Complete 45
CE 96
Complete 46
CNC 97
Complete 47
CNP 98
Complete 48
CDAY 99
Complete 49
CF 100
Complete 50


In [69]:
#Batch 3

bs = 100
rf = 0.0423 #Current 10 year US treasury bond yield
wait_period = 5
verbose = True
startIdx = 101
    
batch_3 = Scrapping(startIdx, bs, rf, wait_period, verbose)

CRL 101
Complete 1
SCHW 102
Complete 2
CHTR 103
Complete 3
CVX 104
Complete 4
CMG 105
Complete 5
CB 106
Complete 6
CHD 107
Complete 7
CI 108
Complete 8
CINF 109
Complete 9
CTAS 110
Complete 10
CSCO 111
Complete 11
C 112
Complete 12
CFG 113
Complete 13
CLX 114
Complete 14
CME 115
Complete 15
CMS 116
Complete 16
KO 117
Complete 17
CTSH 118
Complete 18
CL 119
Complete 19
CMCSA 120
Complete 20
CMA 121
Complete 21
CAG 122
Complete 22
COP 123
Complete 23
ED 124
Complete 24
STZ 125
Complete 25
CEG 126
Complete 26
COO 127
Complete 27
CPRT 128
Complete 28
GLW 129
Complete 29
CSGP 130
Complete 30
COST 131
Complete 31
CTRA 132
Complete 32
CCI 133
Complete 33
CSX 134
Complete 34
CMI 135
Complete 35
CVS 136
Complete 36
DHI 137
Complete 37
DHR 138
Complete 38
DRI 139
Complete 39
DVA 140
Complete 40
DE 141
Complete 41
DAL 142
Complete 42
XRAY 143
Complete 43
DVN 144
Complete 44
DXCM 145
Complete 45
FANG 146
Complete 46
DLR 147
Complete 47
DFS 148
Complete 48
DIS 149
Complete 49
DG 150
Complete 50
DLT

In [70]:
#Batch 4

bs = len(stocks)
rf = 0.0423 #Current 10 year US treasury bond yield
wait_period = 5
verbose = True
startIdx = 201
    
batch_4 = Scrapping(startIdx, bs, rf, wait_period, verbose)

FMC 201
Complete 1
F 202
Complete 2
FTNT 203
Complete 3
FTV 204
Complete 4
FOXA 205
Complete 5
FOX 206
Complete 6
BEN 207
Complete 7
FCX 208
Complete 8
GRMN 209
Complete 9
IT 210
Complete 10
GEHC 211
Complete 11
GEN 212
Complete 12
GNRC 213
Complete 13
GD 214
Complete 14
GE 215
Complete 15
GIS 216
Complete 16
GM 217
Complete 17
GPC 218
Complete 18
GILD 219
Complete 19
GPN 220
Complete 20
GS 221
Complete 21
HAL 222
Complete 22
HIG 223
Complete 23
HAS 224
Complete 24
HCA 225
Complete 25
PEAK 226
Complete 26
HSIC 227
Complete 27
HSY 228
Complete 28
HES 229
Complete 29
HPE 230
Complete 30
HLT 231
Complete 31
HOLX 232
Complete 32
HD 233
Complete 33
HON 234
Complete 34
HRL 235
Complete 35
HST 236
Complete 36
HWM 237
Complete 37
HPQ 238
Complete 38
HUM 239
Complete 39
HBAN 240
Complete 40
HII 241
Complete 41
IBM 242
Complete 42
IEX 243
Complete 43
IDXX 244
Complete 44
ITW 245
Complete 45
ILMN 246
Complete 46
INCY 247
Complete 47
IR 248
Complete 48
PODD 249
Complete 49
INTC 250
Complete 50
ICE

In [71]:
concatenated_df = pd.concat([batch_1, batch_2, batch_3, batch_4], ignore_index=True)

print(concatenated_df)

       Stock Price  Strike Price  Maturity Dividends  Volatility  Risk-free  \
0           102.87          70.0  0.009096    0.0609      0.2177     0.0423   
1           102.87          94.0  0.009096    0.0609      0.2177     0.0423   
2           102.87          95.0  0.009096    0.0609      0.2177     0.0423   
3           102.87          96.0  0.009096    0.0609      0.2177     0.0423   
4           102.87          97.0  0.009096    0.0609      0.2177     0.0423   
...            ...           ...       ...       ...         ...        ...   
41791       188.03         180.0  0.391960    0.0082      0.2000     0.0423   
41792       188.03         185.0  0.391960    0.0082      0.2000     0.0423   
41793       188.03         160.0  1.388537    0.0082      0.2000     0.0423   
41794       188.03         180.0  1.388537    0.0082      0.2000     0.0423   
41795       188.03         185.0  1.388537    0.0082      0.2000     0.0423   

       Call Price Stock Name  
0           33.17   

In [76]:
concatenated_df.to_csv(r'C:\Users\Deepak\Desktop\Dissertation\SNP500_data.csv', index=False, header=True)

In [77]:
concatenated_df

,Stock Price,Strike Price,Maturity,Dividends,Volatility,Risk-free,Call Price,Stock Name
0,102.87,70.0,0.009096,0.0609,0.2177,0.0423,33.17,MMM
1,102.87,94.0,0.009096,0.0609,0.2177,0.0423,8.93,MMM
2,102.87,95.0,0.009096,0.0609,0.2177,0.0423,7.98,MMM
3,102.87,96.0,0.009096,0.0609,0.2177,0.0423,6.95,MMM
4,102.87,97.0,0.009096,0.0609,0.2177,0.0423,6.05,MMM
...,...,...,...,...,...,...,...,...
41791,188.03,180.0,0.391960,0.0082,0.2000,0.0423,17.05,ZTS
41792,188.03,185.0,0.391960,0.0082,0.2000,0.0423,13.80,ZTS
41793,188.03,160.0,1.388537,0.0082,0.2000,0.0423,44.00,ZTS
41794,188.03,180.0,1.388537,0.0082,0.2000,0.0423,30.95,ZTS
